# Parameters and ParameterFrames in bluemira
`Parameters` and `ParameterFrames` are the mechanism bluemira uses to contain
metadata about a given value.
Each `Parameter` must have a unit associated with the value and can have a
source, description and long_name.

The mechanics of the unit system in bluemira are fairly staight forward
It provides an implicit interface to convert units to the base units of bluemira.

In [ ]:
from dataclasses import dataclass

from pint.errors import DimensionalityError

import bluemira.base.constants as const
from bluemira.base.parameter_frame import Parameter, ParameterFrame

## Parameters and Units
First I make a small ParameterFrame and compare the two methods of creating a frame

In [ ]:
@dataclass
class MyParameterFrame(ParameterFrame):
    """A ParameterFrame"""

    R_0: Parameter[float]
    A: Parameter[float]


mypf = MyParameterFrame.from_dict(
    {"A": {"value": 5, "unit": ""}, "R_0": {"value": 8, "unit": "m"}}
)
mypf2 = MyParameterFrame.from_dict(
    {"A": {"value": 5, "unit": ""}, "R_0": {"value": 8, "unit": "m"}}
)

print(mypf)
print(mypf2)
# Both frames equal
assert mypf == mypf2  # noqa: S101

Trying to set a unit with the wrong dimension

In [ ]:
mydiffval = MyParameterFrame.from_dict(
    {"A": {"value": 6, "unit": "m"}, "R_0": {"value": 8, "unit": "m"}}
)

try:
    mypf.update_from_frame(mydiffval)
except DimensionalityError as de:
    print(de)

Changing a value of a parameter with a compatible but different unit

In [ ]:
mypf.update_values({"R_0": {"value": 6000, "unit": "mm"}})

print(mypf)


Accessing the value of a parameter in a different unit

In [ ]:
print(mypf.R_0.value_as("cm"))  # 600

## Raw conversion

In some situations it may be useful to convert a raw value to a different unit.
It is recommended (and enforced within bluemira core) that all conversions however
simplistic are converted this way.

Any possible future base unit changes can be achieved by searching for a given unit.

In [ ]:
print(const.raw_uc(1, "um^3", "m^3"))
# gas flow rate conversion @OdegC
print(const.raw_uc(1, "mol/s", "Pa m^3/s"))
print(const.gas_flow_uc(1, "mol/s", "Pa m^3/s"))
# gas flow rate conversion @25degC
print(const.gas_flow_uc(1, "mol/s", "Pa m^3/s", gas_flow_temperature=298.15))
# boltzmann constant conversion
print(const.raw_uc(1, "eV", "K"))

## Raw Temperature conversion with checks from different units

In [ ]:
try:
    const.to_kelvin(-300)
except ValueError as v:
    print(v)

print(const.to_celsius(10, unit="rankine"))